In [ ]:
!pip install -q transformers accelerate torch torchvision Pillow
!pip install -q qwen-vl-utils==0.0.8
!pip install -q git+https://github.com/huggingface/transformers

In [ ]:
import torch
from transformers import Qwen2_5_VLForConditionalGeneration, AutoTokenizer, AutoProcessor
from qwen_vl_utils import process_vision_info
from PIL import Image
import requests
from io import BytesIO
import json

In [ ]:
model_name = "Qwen/Qwen2.5-VL-7B-Instruct"
model = Qwen2_5_VLForConditionalGeneration.from_pretrained(
    model_name,
    torch_dtype=torch.float16,  # Sử dụng float16 để tiết kiệm bộ nhớ
    device_map="auto",
    trust_remote_code=True
)

In [ ]:
processor = AutoProcessor.from_pretrained(
    model_name,
    trust_remote_code=True
)

In [ ]:
def analyze_skin_condition(image_path_or_url, question=None):
    """
    Phân tích ảnh bệnh da liễu

    Args:
        image_path_or_url: đường dẫn ảnh hoặc URL
        question: câu hỏi cụ thể về tình trạng da
    """

    if question is None:
        question = """Hãy phân tích hình ảnh này về mặt da liễu. Mô tả:
        1. Các tổn thương da có thể quan sát được
        2. Đặc điểm hình thái (màu sắc, kích thước, hình dạng)
        3. Phân bố tổn thương
        4. Gợi ý chẩn đoán phân biệt
        5. Mức độ nghiêm trọng (nếu có thể đánh giá)"""

    # Tạo messages
    messages = [
        {
            "role": "user",
            "content": [
                {
                    "type": "image",
                    "image": image_path_or_url,
                },
                {
                    "type": "text",
                    "text": question
                },
            ],
        }
    ]

In [ ]:
    text = processor.apply_chat_template(
        messages, tokenize=False, add_generation_prompt=True
    )
    image_inputs, video_inputs = process_vision_info(messages)

    inputs = processor(
        text=[text],
        images=image_inputs,
        videos=video_inputs,
        padding=True,
        return_tensors="pt",
    )

In [ ]:
    if torch.cuda.is_available():
        inputs = inputs.to("cuda")
        with torch.no_grad():
        generated_ids = model.generate(
            **inputs,
            max_new_tokens=512,
            do_sample=True,
            temperature=0.3,
            top_p=0.9
        )

    # Decode kết quả
    generated_ids_trimmed = [
        out_ids[len(in_ids):] for in_ids, out_ids in zip(inputs.input_ids, generated_ids)
    ]
    output_text = processor.batch_decode(
        generated_ids_trimmed,
        skip_special_tokens=True,
        clean_up_tokenization_spaces=False
    )

    return output_text[0]

In [ ]:
# Hàm tạo dataset từ CSV của bạn
def create_training_data_from_csv(csv_file_path, output_file="skin_disease_dataset.json"):
    """
    Tạo dataset training từ file CSV của bạn
    """
    import pandas as pd
    import json

    # Đọc file CSV
    df = pd.read_csv(csv_file_path)

    training_data = []

    for idx, row in df.iterrows():
        title = row['title']
        link = row['link']
        detail_images = row['detail_images']

        if pd.notna(detail_images):
            # Xử lý multiple images
            image_urls = [img.strip() for img in detail_images.split(';') if img.strip()]

            for img_url in image_urls[:1]:  # Lấy ảnh đầu tiên cho mỗi bệnh
                training_example = {
                    "disease_name": title,
                    "image_url": img_url,
                    "article_link": link,
                    "prompt": f"Phân tích hình ảnh bệnh da liễu: {title}. Mô tả các đặc điểm lâm sàng quan sát được."
                }
                training_data.append(training_example)

    # Lưu dataset
    with open(output_file, 'w', encoding='utf-8') as f:
        json.dump(training_data, f, ensure_ascii=False, indent=2)

    print(f"Đã tạo dataset với {len(training_data)} mẫu")
    return training_data

In [ ]:
def test_with_sample_image():
    """Test model với ảnh mẫu từ dataset"""

    # Sử dụng ảnh từ dataset của bạn
    sample_image_url = "https://admin.dalieu.vn/image.php?file=https://admin.dalieu.vn/files/uploads/2025/09/1-1431.jpg"

    try:
        print("Đang phân tích ảnh mẫu...")
        result = analyze_skin_condition(
            sample_image_url,
            "Hãy phân tích tổn thương da trong ảnh này. Mô tả đặc điểm lâm sàng và gợi ý chẩn đoán."
        )

        print("\n" + "="*50)
        print("KẾT QUẢ PHÂN TÍCH:")
        print("="*50)
        print(result)
        print("="*50)

    except Exception as e:
        print(f"Lỗi khi phân tích ảnh: {e}")

In [ ]:
test_with_sample_image()

In [ ]:
# Hàm batch processing cho nhiều ảnh
def batch_analyze_images(image_urls, questions=None):
    """
    Phân tích hàng loạt ảnh
    """
    results = []

    for i, img_url in enumerate(image_urls):
        print(f"Đang xử lý ảnh {i+1}/{len(image_urls)}...")

        if questions and i < len(questions):
            question = questions[i]
        else:
            question = "Phân tích tổn thương da trong ảnh này."

        try:
            result = analyze_skin_condition(img_url, question)
            results.append({
                "image_url": img_url,
                "analysis": result
            })
        except Exception as e:
            print(f"Lỗi với ảnh {img_url}: {e}")
            results.append({
                "image_url": img_url,
                "error": str(e)
            })

    return results


In [ ]:
def prepare_finetuning_data(csv_path):
    """
    Chuẩn bị dữ liệu cho fine-tuning
    """
    import pandas as pd

    df = pd.read_csv(csv_path)
    finetuning_data = []

    for idx, row in df.iterrows():
        if pd.notna(row['detail_images']):
            image_urls = [img.strip() for img in row['detail_images'].split(';') if img.strip()]

            # Tạo các cặp prompt-response cho training
            training_example = {
                "conversations": [
                    {
                        "role": "user",
                        "content": [
                            {"type": "image", "image": image_urls[0]},
                            {"type": "text", "text": f"Mô tả các đặc điểm lâm sàng của bệnh {row['title']} trong ảnh này."}
                        ]
                    },
                    {
                        "role": "assistant",
                        "content": f"Đây là hình ảnh minh họa cho bệnh {row['title']}. [Model sẽ học để mô tả chi tiết các tổn thương da]"
                    }
                ]
            }
            finetuning_data.append(training_example)

    return finetuning_data